In [1]:
!pip install pandas geopy folium

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install fuzzywuzzy[speedup]

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import folium
import re
from fuzzywuzzy import process

In [6]:
bus_stops = pd.read_csv('processed_bus_stops.csv', header=0)
stations = pd.read_csv('data\stations_location.csv', header=0)
facilities = pd.read_excel('FINISH_station_facility.xlsx', header=0)
footfall = pd.read_csv('StationFootfall_Total.csv', header=0)
display(bus_stops)
display(stations)
display(facilities)
display(footfall)

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\46842\AppData\Local\Temp\ipykernel_28220\431128874.py:2: SyntaxWarning: invalid escape sequence '\s'
  stations = pd.read_csv('data\stations_location.csv', header=0)


,commonName,lat,lon,is_pure_bus,id,lines
0,Abbeville Road,51.452640,-0.142490,1,490004889E,"355, 50"
1,Abbeville Road,51.452730,-0.142900,1,490004889W,"355, 50"
2,Abbey Lane,51.533780,-0.007180,1,490003025W,"108, 25, 276, 425, D8, N205, N25, N8"
3,Abbey Road,51.415980,-0.185750,1,490007938E,"131, 152, 200, 219, 57, 655, N155"
4,Abbey Road,51.416110,-0.185360,1,490007938F,"131, 152, 200, 219, 57, 655, N155"
...,...,...,...,...,...,...
21965,Zangwill Road,51.472040,0.045300,1,490013275S,386
21966,Zangwill Road,51.472280,0.045130,1,490013275N,386
21967,Zig Zag Road,51.258173,-0.322678,1,40004408016A,465
21968,Zig Zag Road,51.258392,-0.322871,1,40004408223A,465


,line_id,line_name,mode_name,station_id,station_name,lat,lon,station_modes
0,bakerloo,Bakerloo,tube,940GZZLUBST,Baker Street Underground Station,51.522883,-0.157130,['tube']
1,bakerloo,Bakerloo,tube,940GZZLUCHX,Charing Cross Underground Station,51.507410,-0.127277,['tube']
2,bakerloo,Bakerloo,tube,940GZZLUEAC,Elephant & Castle Underground Station,51.494536,-0.100606,['tube']
3,bakerloo,Bakerloo,tube,940GZZLUEMB,Embankment Underground Station,51.507058,-0.122666,['tube']
4,bakerloo,Bakerloo,tube,940GZZLUERB,Edgware Road (Bakerloo) Underground Station,51.520299,-0.170150,['tube']
...,...,...,...,...,...,...,...,...
584,windrush,Windrush,overground,910GSYDENHM,Sydenham Rail Station,51.427248,-0.054244,"['national-rail', 'overground']"
585,windrush,Windrush,overground,910GWAPPING,Wapping Rail Station,51.504388,-0.055931,"['bus', 'overground']"
586,windrush,Windrush,overground,910GWCHAPEL,Whitechapel Rail Station,51.519469,-0.059757,['overground']
587,windrush,Windrush,overground,910GWCROYDN,West Croydon Rail Station,51.378428,-0.102585,"['national-rail', 'overground']"


,Grid,Station Name,Internal interchange or short walk,Level access boarding point,Boarding ramp available,Mini ramp may be available,Lift access,Step-free National Rail interchange,Main bus interchange,Accessible River services,...,Platform-train gap: Green (0–50mm),Platform-train gap: Orange (51–120mm),Platform-train gap: Red (Over 120mm),Note,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,C8,Abbey Road,NaN,NaN,NaN,NaN,1,NaN,1.0,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,elevator regular
1,E9,Abbey Wood,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,Elizabeth line and Thameslink,NaN,NaN,NaN,NaN,2,elevator small
2,D2,Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Step-free platform interchange requires a 50m ...,NaN,NaN,NaN,NaN,NaN,NaN
3,C2,Acton Main Line,NaN,NaN,2.0,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,D2,Acton Town,NaN,NaN,1.0,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,A8,Woodford,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Step-free platform interchange requires a 130m...,NaN,NaN,NaN,NaN,NaN,NaN
310,F8,Woodside,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,A5,Woodside Park,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Step-free platform interchange requires 600m v...,NaN,NaN,NaN,NaN,NaN,NaN
312,E9,Woolwich,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Station,EntryTapCount,ExitTapCount,TotalFlow
0,Kings Cross St Pancras,181536467,183702980,365239447
1,Victoria,159023832,164248825,323272657
2,London Bridge,148511293,144126893,292638186
3,Oxford Circus,134336368,153923312,288259680
4,Stratford,141428793,145494472,286923265
...,...,...,...,...
431,Emerson Park,563251,378782,942033
432,Unknown,0,824606,824606
433,Twyford,365329,300030,665359
434,Iver,295719,269612,565331


In [7]:
def clean_name(name):
    if pd.isna(name):
        return ""
    name = name.lower()
    name = re.sub(r"[^\w\s]", "", name)
    name = re.sub(r"\s+", " ", name).strip()
    return name

footfall['clean_station'] = footfall['Station'].apply(clean_name)
facilities['clean_station'] = facilities['Station Name'].apply(clean_name)
stations['clean_station'] = stations['station_name'].apply(clean_name)

facility_names = facilities['clean_station'].unique()
station_names = stations['clean_station'].unique()

facility_match = {}
for name in footfall['clean_station']:
    match, score = process.extractOne(name, facility_names)
    if score >= 85:
        facility_match[name] = match
    else:
        facility_match[name] = None

station_match = {}
for name in footfall['clean_station']:
    match, score = process.extractOne(name, station_names)
    if score >= 85:
        station_match[name] = match
    else:
        station_match[name] = None

footfall['matched_facility'] = footfall['clean_station'].map(facility_match)
footfall['matched_station'] = footfall['clean_station'].map(station_match)

facilities_unique = facilities.drop_duplicates(subset='clean_station')
stations_unique = stations.drop_duplicates(subset='clean_station')

merged = footfall.merge(facilities_unique, left_on='matched_facility', right_on='clean_station', how='left', suffixes=('', '_facility'))
merged = merged.merge(stations_unique, left_on='matched_station', right_on='clean_station', how='left', suffixes=('', '_station'))

print("Merged data:")
display(merged[['Station', 'TotalFlow', 'Station Name', 'Lift access', 'lat', 'lon']].head())

Merged data:


,Station,TotalFlow,Station Name,Lift access,lat,lon
0,Kings Cross St Pancras,365239447,King’s Cross & St Pancras International,1,51.530663,-0.123194
1,Victoria,323272657,Victoria,1,51.496359,-0.143102
2,London Bridge,292638186,London Bridge,1,51.505721,-0.088873
3,Oxford Circus,288259680,Oxford Circus,NaN,51.515224,-0.141903
4,Stratford,286923265,Stratford,1,51.541806,-0.003458


In [8]:
no_stepfree = merged[merged['Lift access'].isna()]
no_stepfree = pd.merge(
    no_stepfree,
    stations[['station_name', 'lat', 'lon']],
    left_on='Station',
    right_on='station_name',
    how='left'
)
no_stepfree = no_stepfree.dropna(subset=['clean_station_station'])
display(no_stepfree)

,Station,EntryTapCount,ExitTapCount,TotalFlow,clean_station,matched_facility,matched_station,Grid,Station Name,Internal interchange or short walk,...,mode_name,station_id,station_name_x,lat_x,lon_x,station_modes,clean_station_station,station_name_y,lat_y,lon_y
0,Oxford Circus,134336368,153923312,288259680,oxford circus,oxford circus,oxford circus underground station,C4,Oxford Circus,NaN,...,tube,940GZZLUOXC,Oxford Circus Underground Station,51.515224,-0.141903,['tube'],oxford circus underground station,NaN,NaN,NaN
1,Bond Street,84974705,86049202,171023907,bond street,bond street,bond street,C4,Bond Street,NaN,...,elizabeth-line,910GBONDST,Bond Street,51.513690,-0.149517,['elizabeth-line'],bond street,Bond Street,51.51369,-0.149517
2,Green Park,71442448,83967993,155410441,green park,green park,green park underground station,D4,Green Park,NaN,...,tube,940GZZLUGPK,Green Park Underground Station,51.506947,-0.142787,['tube'],green park underground station,NaN,NaN,NaN
3,Leicester Square,77004999,72305369,149310368,leicester square,None,leicester square underground station,NaN,NaN,NaN,...,tube,940GZZLULSQ,Leicester Square Underground Station,51.511386,-0.128426,['tube'],leicester square underground station,NaN,NaN,NaN
4,Bank,70693399,72228257,142921656,bank,bank,embankment underground station,D6,Bank,1.0,...,tube,940GZZLUEMB,Embankment Underground Station,51.507058,-0.122666,['tube'],embankment underground station,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,Headstone Lane,1090063,966641,2056704,headstone lane,headstone lane,headstone lane rail station,A3,Headstone Lane,NaN,...,overground,910GHEDSTNL,Headstone Lane Rail Station,51.602649,-0.357220,['overground'],headstone lane rail station,NaN,NaN,NaN
238,Barking Riverside,972552,845794,1818346,barking riverside,barking riverside,barking riverside,C9,Barking Riverside,NaN,...,overground,910GBKRVS,Barking Riverside,51.519530,0.115960,['overground'],barking riverside,Barking Riverside,51.51953,0.115960
239,Chigwell,943076,810709,1753785,chigwell,None,chigwell underground station,NaN,NaN,NaN,...,tube,940GZZLUCWL,Chigwell Underground Station,51.617916,0.075041,"['bus', 'tube']",chigwell underground station,NaN,NaN,NaN
240,Roding Valley,509201,439353,948554,roding valley,roding valley,roding valley underground station,A8,Roding Valley,NaN,...,tube,940GZZLURVY,Roding Valley Underground Station,51.617199,0.043647,['tube'],roding valley underground station,NaN,NaN,NaN


In [9]:
def min_bus_distance_single(station_lat, station_lon, bus_df):
    return min([
        geodesic((station_lat, station_lon), (bus_lat, bus_lon)).meters
        for bus_lat, bus_lon in zip(bus_df['lat'], bus_df['lon'])
    ])

distances = []
for _, row in no_stepfree.iterrows():
    dist = min_bus_distance_single(row['lat_x'], row['lon_x'], bus_stops)
    distances.append(dist)
no_stepfree['NearestBusDistance_m'] = distances

In [10]:
top_busiest = no_stepfree.sort_values('TotalFlow', ascending=False).head(200)
crisis_stations = top_busiest[top_busiest['NearestBusDistance_m'] > 200]

print("Crisis Stations (Top 200 Busiest, No Step-Free, Bus > 200m):")
print(crisis_stations[['Station', 'TotalFlow', 'NearestBusDistance_m']])

Crisis Stations (Top 200 Busiest, No Step-Free, Bus > 200m):
           Station  TotalFlow  NearestBusDistance_m
14         Barking   85328204            296.936955
22   Covent Garden   66404564            303.712840
70    Barons Court   27227345            396.711034
124         Epping   15589729           4143.881085
177       Cheshunt    8467080           1974.562710
185  Acton Central    8175569            241.086786


In [12]:
map_center = [51.5074, -0.1278]
map_crisis = folium.Map(location=map_center, zoom_start=11)

for _, row in crisis_stations.iterrows():
    folium.Marker(
        location=[row['lat_x'], row['lon_x']],
        popup=(f"Station: {row['Station'].title()}<br>"
               f"Total Flow: {row['TotalFlow']:.0f}<br>"
               f"Nearest Bus Distance: {row['NearestBusDistance_m']:.1f} m"),
        icon=folium.Icon(color='red', icon='exclamation-sign')
    ).add_to(map_crisis)

map_crisis

In [ ]:
map_crisis.save('crisis_stations_map.html')

In [11]:
map_center = [51.5074, -0.1278]
no_stepfree_map = folium.Map(location=map_center, zoom_start=11)

def get_color(flow):
    if flow > no_stepfree['TotalFlow'].quantile(0.75):
        return 'red'
    elif flow > no_stepfree['TotalFlow'].quantile(0.5):
        return 'orange'
    else:
        return 'green'

for _, row in no_stepfree.iterrows():
    folium.CircleMarker(
        location=[row['lat_x'], row['lon_x']],
        radius=6,
        color=get_color(row['TotalFlow']),
        fill=True,
        fill_opacity=0.7,
        popup=(f"<b>Station:</b> {row['Station'].title()}<br>"
               f"<b>Total Flow:</b> {row['TotalFlow']:.0f}"
               + (f"<br><b>Nearest Bus Distance:</b> {row['NearestBusDistance_m']:.1f} m"
                  if 'NearestBusDistance_m' in row else "")),
    ).add_to(no_stepfree_map)

no_stepfree_map

In [ ]:
no_stepfree_map.save('no_stepfree_stations_map.html')

In [13]:
import folium
import branca.colormap as cm

# create a linear colour mapping
min_flow = no_stepfree['TotalFlow'].min()
max_flow = no_stepfree['TotalFlow'].max()
colormap = cm.linear.YlOrRd_09.scale(min_flow, max_flow)
colormap.caption = 'Station Total Flow'

# create map
map_center = [51.5074, -0.1278]
no_stepfree_map = folium.Map(location=map_center, zoom_start=11)

# add site circle
for _, row in no_stepfree.iterrows():
    folium.CircleMarker(
        location=[row['lat_x'], row['lon_x']],
        radius=6,
        color=colormap(row['TotalFlow']),
        fill=True,
        fill_opacity=0.7,
        popup=(f"<b>Station:</b> {row['Station'].title()}<br>"
               f"<b>Total Flow:</b> {row['TotalFlow']:.0f}"
               + (f"<br><b>Nearest Bus Distance:</b> {row['NearestBusDistance_m']:.1f} m"
                  if 'NearestBusDistance_m' in row else ""))
    ).add_to(no_stepfree_map)

# add captions
colormap.add_to(no_stepfree_map)

no_stepfree_map